In [1]:
# import time

# start = time.time()
# print("hello")
# end = time.time()
# print(end - start)

In [2]:
import pandas as pd
import numpy as np
from random import randint
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import normalized_mutual_info_score as nmi_score

In [3]:
train = pd.read_csv("./mnist-in-csv/mnist_train.csv")
test = pd.read_csv("./mnist-in-csv/mnist_test.csv")

In [4]:
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train_data = train.drop("label",axis=1)
train_data = train_data.values
train_labels = train["label"].values

test_data = test.drop("label",axis=1)
test_data = test_data.values
test_labels = test["label"].values

"""print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)
"""
print(train_labels)
"""t0 = train_data[0]
l0 = train_labels[0]
print(t0,l0)"""

[5 0 4 ..., 5 6 8]


't0 = train_data[0]\nl0 = train_labels[0]\nprint(t0,l0)'

In [6]:
tr_data_labels = np.c_[train_data,train_labels]
np.random.shuffle(tr_data_labels)
tr_data_labels

train_labels = tr_data_labels[:,784]
train_data = np.delete(tr_data_labels, 784, 1)
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)
train_labels


(60000, 784)
(60000,)
(10000, 784)
(10000,)


array([8, 1, 5, ..., 6, 3, 7])

## K-Means

In [8]:
# initialize means to 10 random points
means_indices = np.random.randint(low=0,high=60000,size=10)
#means_indices = [36,31,28,24,21,19,18,26,34,30]
means_new = np.zeros((10,784))
means_old = np.array([train_data[i] for i in means_indices])
for i in means_indices:
    print(train_labels[i]) 

2
8
8
3
5
7
7
0
9
4


In [9]:
eps_dist = 1
dist = np.linalg.norm(means_new-means_old)
dist = 100
#dist2 = 0


In [10]:
classes = []
mean_dists = [[0 for i in range(60000)] for j in range(10)]
pbar = tqdm(total = 200+1)
while dist >= eps_dist:
    
    print(dist,eps_dist,np.count_nonzero(means_old))
    b = np.zeros((10,60000))
    mean_dists = [[0 for i in range(60000)] for j in range(10)]
    
    for i in range(10):
        for j in range(60000):
            mean_dists[i][j] = np.linalg.norm(train_data[j]-means_old[i])
    mean_dists = np.array(mean_dists)
    
    for j in range(60000):
        b[np.argmin(mean_dists[:,j])][j] = 1
        
    b_sum = np.sum(b,axis=1)
    
    for i in range(10):
        for j in range(60000):
            means_new[i] = np.add(means_new[i],b[i][j]*train_data[j])
        means_new[i] = np.divide(means_new[i],b_sum[i])
    
    #dist = np.linalg.norm(means_new-means_old)
    diff = means_new-means_old
    lst = []
    for i in range(10):
        lst.append(np.linalg.norm(diff[i]))
    dist = np.max(np.array(lst))
    means_old = means_new
    means_new =  np.zeros((10,784))
    pbar.update(1)
print(dist,eps_dist)

100 1 1425
1563.3924394930261 1 6156
369.412514582816 1 6115
251.73825689685378 1 6134
306.682181811133 1 6082
201.15047457991395 1 6061
140.7468006218947 1 6036
109.36798893914538 1 6011
91.64589064917901 1 5990
86.2444620679993 1 5995
74.4697653264237 1 5994
63.293046770840846 1 5991
47.0428951919667 1 5993
32.14531048243017 1 5991
27.48278271813446 1 5987
25.702141266855296 1 5993
20.05255112402001 1 5992
18.619670077647026 1 5996
17.091060128276418 1 5991
14.480569097785198 1 5986
12.171371494115839 1 5981
11.375317608075958 1 5982
12.575204423201155 1 5980
10.610844421898543 1 5983
11.78061923046568 1 5985
12.630370832397597 1 5984
12.775789565395822 1 5989
13.272979117405368 1 5990
11.668567371132218 1 5990
12.19314731900215 1 5987
11.26895852089021 1 5986
9.443896929680449 1 5988
8.143663262467149 1 5989
8.599539597938758 1 5989
8.847747404936067 1 5992
9.802375710244068 1 5989
8.875316841520705 1 5992
7.527507925878601 1 5992
6.696701706715197 1 5993
5.531534709455088 1 5996
4.

In [11]:
N = 60000
class_data = [[] for i in range(10)]
for i in range(N):
    class_data[train_labels[i]].append(train_data[i])

In [12]:
cnt = 0
for i in range(10):
    cnt += len(class_data[i])
    print(len(class_data[i]))

5923
6742
5958
6131
5842
5421
5918
6265
5851
5949


In [13]:
lst_train1 = [0 for i in range(N)]

In [14]:
for i in range(60000):
    lst_train1[i] = tuple(train_data[i])

In [ ]:
b = [[0 for j in range(N)] for i in range(10)]
dis = [0 for j in range(N)]
for i in range(10):
    for j in range(N):
        dis[j] = np.linalg.norm(train_data[j]-means_old[i])
    b[]

In [35]:
final_labels = [-1 for i in range(10)]

for i in tqdm(range(10)):
    counts = [0 for j in range(10)]
    for j in range(N):
        if b[i][j] == 1:
            counts[train_labels[j]] += 1
    counts = np.array(counts)
    print(counts)
    final_labels[i] = counts.argmax()

[  29 2978  369   84  230  712  191  236  390   77]
[ 584    6  141 1230   13 1833  130    3 3106   44]
[  39    9 4193  205   15   14   88   30   43    5]
[  86    4  227   68 2200  190  534  574  136 1410]
[ 233   10  163   61  128  138 4626    2   53    8]
[   0 3713  320  369   98  118  243  218  278  139]
[  13    9   39  161 1614  303    1 2392  208 2455]
[4679    0   55   34    4   63   72   13   29   29]
[ 233    6  391 3887    0 1783   32    3 1374   88]
[  27    7   60   32 1540  267    1 2794  234 1694]


In [36]:
final_labels

[1, 8, 2, 4, 6, 1, 9, 0, 3, 7]

In [37]:
# assigning label 5 to cluster 0 as it has more 5's than cluster 5 (counting of cluster is done from 0)
final_labels[0] = 5

In [38]:
final_labels

[5, 8, 2, 4, 6, 1, 9, 0, 3, 7]

In [50]:
for i in range(10):
    for j in range(60000):
        mean_dists[i][j] = np.linalg.norm(train_data[j]-means_old[i])
mean_dists = np.array(mean_dists)
    
for j in range(60000):
    b[np.argmin(mean_dists[:,j])][j] = 1
        
b_sum = np.sum(b,axis=1)

In [51]:
mis_pred = 0
for i in tqdm(range(10)):
    counts = [0 for j in range(10)]
    for j in range(N):
        if b[i][j] == 1:
            counts[train_labels[j]] += 1
    for k in range(10):
        mis_pred += counts[k]
    mis_pred -= counts[final_labels[i]]

In [55]:
print("\nTrain accuracy: ",100-(mis_pred/N)*100,"%\n")


Train accuracy:  53.92 %



In [53]:
test_pred = [0 for i  in range(10000)]
for i in range(10000):
    test_dists = [0]*10
    for j in range(10):
        test_dists[j] = np.linalg.norm(means_old[j]-test_data[i])
    test_pred[i] = final_labels[np.array(test_dists).argmin()]

test_acc = 0
for i in range(10000) :
    if test_pred[i] == test_labels[i]:
        test_acc = test_acc + 1

In [56]:
print("\nTest accuracy: ",(test_acc/10000)*100,"%\n")


Test accuracy:  54.74 %



In [65]:
res_dist = [0]*10
res_dmtr = [0]*10
res_acc = [0]*10
for i in tqdm(range(10)):
    indices = [j for j, n in enumerate(b[i]) if n == 1]
    dists_mean = np.zeros((len(indices),len(indices)))
    sum_squares = 0
    for j in tqdm(range(len(indices))):
        if train_labels[indices[j]] == final_labels[i]:
            res_acc[i] += 1
        sum_squares += np.power(np.linalg.norm(means_old-train_data[indices[j]]),2)
        for k in range(j,len(indices)):
            dists_mean[j][k] = np.linalg.norm(train_data[indices[k]] - train_data[indices[j]])
    res_dmtr[i] = np.max(dists_mean)
    res_dist[i] = sum_squares
    res_acc[i] /= len(indices)

In [67]:
res_tab = pd.DataFrame({"Label":final_labels,
                        "Accuracy":res_acc,
                        "Diameter":res_dmtr,
                        "Sq.Distance":res_dist})
res_tab

,Label,Accuracy,Diameter,Sq.Distance
0,5,0.134441,3222.821590,1.942741e+11
1,8,0.438020,3805.877428,3.028566e+11
2,2,0.903469,3855.856325,2.302895e+11
3,4,0.404709,3786.763130,2.585273e+11
4,6,0.853191,3687.544847,2.373480e+11
5,1,0.675582,3022.513027,1.848557e+11
6,9,0.341064,3529.545296,2.772344e+11
7,0,0.939936,3971.084613,2.897588e+11
8,3,0.498525,3805.211689,3.651706e+11
9,7,0.419772,3526.387103,2.651400e+11


In [69]:
size = [1,2,3]

In [70]:
size[:-1]

[1, 2]

In [71]:
for sizes in size[:-1]:
    print(sizes)

1
2


In [11]:
def k_means(train_data, train_labels, eps_dis = 1, d = 784, k = 10):
    # initialize means to 10 random points
    N = len(train_data)
    means_indices = np.random.randint(low=0,high=len(train_data),size=k)
    #means_indices = [36,31,28,24,21,19,18,26,34,30]
    means_new = np.zeros((k,d))
    means_old = np.array([train_data[i] for i in means_indices])
    for i in means_indices:
        print(train_labels[i]) 

    eps_dist = eps_dis
    diff = means_new-means_old
    lst = []
    for i in range(k):
        lst.append(np.linalg.norm(diff[i]))
    dist = np.max(np.array(lst))
    #dist = 100
    #dist2 = 0

    classes = []
    mean_dists = [[0 for i in range(len(train_data))] for j in range(k)]
    pbar = tqdm(total = 200+1)

    cntr = 0
    while dist >= eps_dist:
        cntr += 1
        b = np.zeros((k,len(train_data)))
        mean_dists = [[0 for i in range(len(train_data))] for j in range(k)]

        for i in range(k):
            for j in range(len(train_data)):
                mean_dists[i][j] = np.linalg.norm(train_data[j]-means_old[i])
        mean_dists = np.array(mean_dists)

        for j in range(len(train_data)):
            b[np.argmin(mean_dists[:,j])][j] = 1

        b_sum = np.sum(b,axis=1)

        for i in range(k):
            for j in range(len(train_data)):
                means_new[i] = np.add(means_new[i],b[i][j]*train_data[j])
            means_new[i] = np.divide(means_new[i],b_sum[i])

        #dist = np.linalg.norm(means_new-means_old)
        diff = means_new-means_old
        lst = []
        for i in range(k):
            lst.append(np.linalg.norm(diff[i]))
        dist = np.max(np.array(lst))
        means_old = means_new
        means_new =  np.zeros((k,d))
        pbar.update(1)
        print("Iter %d: dist = " %(cntr),dist)#eps_dist,np.count_nonzero(means_old))

    #print(dist,eps_dist)
    # finding diameters of clusters
    print("computing diameters...")
    print("\tcomputing distances from the mean...")
    diams = [0 for i in range(k)]
    b = np.zeros((k,len(train_data)))
    for i in tqdm(range(k)):
            for j in range(len(train_data)):
                mean_dists[i][j] = np.linalg.norm(train_data[j]-means_old[i])
    mean_dists = np.array(mean_dists)

    print("\tfinding diameters...")
    for j in range(len(train_data)):
            b[np.argmin(mean_dists[:,j])][j] = 1
    for i in tqdm(range(k)):
        for j in tqdm(range(N)):
            for l in range(N):
                if b[i][j] == 1 and b[i][l] == 1:
                    t = np.linalg.norm(train_data[j]-train_data[l])
                    if diams[i] < t:
                        diams[i] = t
                
    
    # predicting labels
    print("Done.\n\npredicting labels...")
    final_labels = [0 for i in range(N)]
    for j in tqdm(range(N)):
        tmp = np.inf
        lbl = -1
        for i in range(k):
            dis = np.linalg.norm(train_data[j]-means_old[i])
            if dis < tmp:
                tmp = dis
                lbl = i
        final_labels[j] = lbl
        
    print("Done.\n\ncomputing square_distances...")
    val = [0 for i in range(k)]
    for i in tqdm(range(N)):
        val[int(final_labels[i])] += np.power(np.linalg.norm(train_data[i]-means_old[int(final_labels[i])]),2)
    
    print("means: ",means_old)
    print("sq_dists: ",val)
    print("diameters: ",diams)
    return nmi_score(train_labels, final_labels)

In [12]:
nmi = k_means(train_data, train_labels)
print("NMI: ",nmi)

9
0
2
8
7
7
0
2
9
9


Iter 1: dist =  1530.50907039
Iter 2: dist =  304.970693913
Iter 3: dist =  189.770044747
Iter 4: dist =  140.597430557
Iter 5: dist =  100.437472537
Iter 6: dist =  88.8845618615
Iter 7: dist =  86.7556285471
Iter 8: dist =  70.6846226195
Iter 9: dist =  59.6879824219
Iter 10: dist =  55.1564744933
Iter 11: dist =  60.5600495117
Iter 12: dist =  65.8249755518
Iter 13: dist =  76.7226092808
Iter 14: dist =  86.5618309226
Iter 15: dist =  89.4197537094
Iter 16: dist =  76.5869689521
Iter 17: dist =  68.104441824
Iter 18: dist =  57.1979620515
Iter 19: dist =  55.6366976875
Iter 20: dist =  53.8734460903
Iter 21: dist =  48.3879441998
Iter 22: dist =  37.4277516376
Iter 23: dist =  29.5182110453
Iter 24: dist =  24.6766928267
Iter 25: dist =  21.737373657
Iter 26: dist =  18.5158825005
Iter 27: dist =  14.4644918599
Iter 28: dist =  13.4981197677
Iter 29: dist =  11.892904965
Iter 30: dist =  8.73935028181
Iter 31: dist =  7.51993010754
Iter 32: dist =  6.72980099177
Iter 33: dist =  4.6

	finding diameters...


Done.

predicting labels...


Done.

computing square_distances...


means:  [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
sq_dists:  [18122200520.356575, 9218415287.6968861, 15771787775.787598, 21927247376.751011, 10267837338.392799, 20480380042.249371, 8844320232.0576534, 15110871297.341501, 9081122652.7205734, 24168430228.252979]
diameters:  [3619.4489912139943, 3915.3889972772822, 3687.5448471849127, 3805.2116892493641, 3181.0572141978209, 3526.3871029709712, 3886.0887792226263, 3855.8563251241612, 3022.5130272672109, 3727.9840396654063]
NMI:  0.490872422144


#### Note: Code can be improved.
